This is a setup/test/demonstration notebook for the `AutoExposure` effect in Scopesim. The notebook uses the `irdb/METIS` configuration. The observed source is blank sky, except for the last example where a star of 0 mag is used (Vega).

In [ ]:
import scopesim as sim
sim.bug_report()

# Edit this path if you have a custom install directory, otherwise comment it out.
sim.rc.__config__["!SIM.file.local_packages_path"] = "../../../../"

If you haven't got the instrument packages yet, uncomment the following cell. 

In [ ]:
# sim.download_packages(["METIS", "ELT", "Armazones"])

# Imaging LM-band

In [ ]:
cmd = sim.UserCommands(use_instrument="METIS", set_modes=["img_lm"])

In [ ]:
metis = sim.OpticalTrain(cmd)

In [ ]:
src = sim.source.source_templates.empty_sky()

In [ ]:
metis.observe(src)

In [ ]:
outimg = metis.readout()[0][1].data 
outimg /= sim.utils.from_currsys(metis['summed_exposure'].meta['ndit'])

full_well = sim.utils.from_currsys("!DET.full_well")
print("\nResult\n======")
print("Maximum value in readout (per DIT): {:8.1f}".format(outimg.max()))
print("Detector full well:                 {:8.1f}".format(full_well))
print("Fill fraction:                      {:8.1f} per cent".format(100 * outimg.max()/ full_well))

Exposure time can be changed with an argument to `metis.readout()`:

In [ ]:
outimg = metis.readout(exptime = 1000)[0][1].data 
outimg /= sim.utils.from_currsys(metis['summed_exposure'].meta['ndit'])

full_well = sim.utils.from_currsys("!DET.full_well")
print("\nResult\n======")
print("Maximum value in readout (per DIT): {:8.1f}".format(outimg.max()))
print("Detector full well:                 {:8.1f}".format(full_well))
print("Fill fraction:                      {:8.1f} per cent".format(100 * outimg.max()/ full_well))

# Imaging N-band

In [ ]:
cmd = sim.UserCommands(use_instrument="METIS", set_modes=['img_n'])

In [ ]:
metis = sim.OpticalTrain(cmd)

In [ ]:
metis.observe(src)

In [ ]:
outimg = metis.readout()[0][1].data 
outimg /= sim.utils.from_currsys(metis['summed_exposure'].meta['ndit'])

full_well = sim.utils.from_currsys("!DET.full_well")
print("\nResult\n======")
print("Maximum value in readout (per DIT): {:9.1f}".format(outimg.max()))
print("Detector full well:                 {:9.1f}".format(full_well))
print("Fill fraction:                      {:9.1f} per cent".format(100 * outimg.max()/ full_well))

# Long-slit spectroscopy

In [ ]:
cmd = sim.UserCommands(use_instrument="METIS", set_modes=['lss_l'])

In [ ]:
metis = sim.OpticalTrain(cmd)

In [ ]:
metis.observe(src)

In [ ]:
outimg = metis.readout(exptime=3600.)[0][1].data 
outimg /= sim.utils.from_currsys(metis['summed_exposure'].meta['ndit'])
full_well = sim.utils.from_currsys("!DET.full_well")

print("\nResult\n======")
print("Maximum value in readout (per DIT): {:8.1f}".format(outimg.max()))
print("Detector full well:                 {:8.1f}".format(full_well))
print("Fill fraction:                      {:8.1f} per cent".format(100 * outimg.max()/ full_well))

# What happens when the source saturates the detector?
Use N-band imaging of Vega. DIT is automatically set to the minimum possible value, but the centre of the star still saturates the detector. In the final image, the star's profile is capped at the full well of the detector.

In [ ]:
cmd = sim.UserCommands(use_instrument="METIS", set_modes=["img_n"])

In [ ]:
metis = sim.OpticalTrain(cmd)

In [ ]:
src = sim.source.source_templates.star()

In [ ]:
metis.observe(src)

In [ ]:
outimg = metis.readout()[0][1].data
outimg /= sim.utils.from_currsys(metis['summed_exposure'].meta['ndit'])
full_well = sim.utils.from_currsys("!DET.full_well")

print("\nResult\n======")
print("Maximum value in readout (per DIT): {:9.1f}".format(outimg.max()))
print("Detector full well:                 {:9.1f}".format(full_well))
print("Fill fraction:                      {:9.1f} per cent".format(100 * outimg.max()/ full_well))

Plot a cut through the star to show how its peak saturates the detector.

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
plt.plot(outimg[950:1100, 1024])

In [ ]:
npix = (outimg >= full_well).sum()
print("Number of saturated pixels:", npix)